# GRID SEARCH BY CROSS VALIDATION 
# Analyse de la relation entre l'âge et la maturité des dents

Le dataset contient l'état, déterminé par un expert médical, de plusieurs dents par patient. 

8 dents sont étudiées (VAL_I1, VAL_I2, VAL_C1, VAL_P1, VAL_P2, VAL_M1, VAL_M2, VAL_M3), chacune de ses dents ont un score de maturité (0, 1, A, B, C, D, E, F, G, H), 0 étant l'état le moins mature, 8 étant l'état le plus mature. 

Le dataset contient aussi l'âge et le sexe du patient. 

Nous considérons l'âge comme la variable dépendente dans cette étude, les autres variables (hors *ID*) seront explicatives. 

#### Imports and Settings

In [ ]:
# IMPORTS ------

import pandas as pd
import numpy as np
import nbformat

    #* Sklearn ---
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error

from sklearn import preprocessing
from sklearn.decomposition import PCA

from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MaxAbsScaler

    #* Graphics --- 
pd.options.plotting.backend = "plotly" # must have plotly installed + nbformat 
import plotly
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio
pio.renderers.default = "notebook"

In [ ]:
# Settings ---
PATH= "dataset.csv"
KNNImputerNeighbors=5

## 1. Charger le dataset dans un dataframe Pandas

In [ ]:
# LECTURE ------
dataset=pd.read_csv(PATH, sep=";")

# CAST TO CATEGORY ------
for i in dataset.columns[3:]:
    dataset[i]= dataset[i].astype("category")

## Supprimer la colonne ID.

In [ ]:
# DROP ID ------
dataset.drop(labels=['ID'], axis=1,inplace=True)

Aperçu de la table: 

In [ ]:
dataset.head()

## Séparer les colonnes en :
### a) X pour le sexe et la maturité de chacune des dents

In [ ]:
X=dataset.loc[:,dataset.columns!="PAT_AGE"] # separer X

### b) Y pour l’âge qui correspond à la colonne à prédire

In [ ]:
y=dataset["PAT_AGE"] # separer Y

## Remplacer les lettres pour la maturité dentaire par des valeurs : [A, 2], [B, 3], [C, 4], [D, 5], [E, 6], [F, 7], [G, 8] et [H, 9]. Les observations doivent toutes être du type numérique.

### 2.1 Représentation numérique pour des variables catégoriques.

Nous exploitons le caractère odinaire des *scores de maturité*, ainsi, nous assignons une valeur numérique à chaque score de maturité dentale. 

En adoptant cette approche, nous supposons que **l'augmentation de la gravité de la maturité des dents est linéaire** entre les différents scores. En d'autres termes, la différence d'état de la dent (delta de gravité observée) est la même entre *A* et *B* qu'entre *B* et *C*. Cette supposition est importante. 

De ce fait, nous accordons des valeurs de 0 à 9 à chaque état dentaire (0,1,A,B...) respectivement. 

Le résultat est le suivant, ce *mapping* sera utilisé:

In [ ]:
# CATEGORIES ORDINALES ------
categories = ["0", "1", "A", "B", "C", "D", "E", "F", "G", "H"]
categoriesDict = {ith:label for ith,label in enumerate(categories)} # 0:0 1:1 2:A ... 

# NAME OF ORDINAL VARS ------
categoric=dataset.columns[2:]
nCategoric=len(categoric)  # Size

# *Print --- 
print("Replace categories given this rule :" ,categoriesDict) # only for illustration, OrdinalEncoder does the work.

In [ ]:
# SCORES (ETATS) SONT PARTAGES POUR TOUTES LES DENTS -----
catMatrix=[]
for i in range(0,nCategoric,1):
    catMatrix.append(categories) # Matrice (n_colonnes,n_categories)

In [ ]:
# PREPROCESSING CAT TO NUM ------
ordEncoder = preprocessing.OrdinalEncoder(categories=catMatrix, unknown_value= np.nan,handle_unknown='use_encoded_value') # créer le encoder. 
encoded=ordEncoder.fit_transform(X[categoric]) # apply transformation. 
EncodedDataset= pd.DataFrame(encoded,columns=categoric) # To DF
EncodedDataset=pd.concat([X["PAT_SEX"].reset_index(drop=True), EncodedDataset], axis=1) # Add non transformed vars. 

Toutes les colonnes sont numériques: 

In [ ]:
# ALL COLUMNS ARE NUMERIC ------
EncodedDataset.dtypes 

## Remplacer les données manquantes de la feature VAL_M3 par zéro (dent absente).

La colonne VAL_M3 est la moins complète, c'est à dire, qu'elle a le plus grand nombre de valeur manquantes. 

Nous allons remplacer toutes ses valeurs par 0, en d'autres termes, nous traduisons la valeur nulle pour cette dent comme représentant le fait que la dent n'a pas encore poussé. 

In [ ]:
nMissingM3= pd.isna(EncodedDataset["VAL_M3"]).sum()
print(f"Pourcentage de valeur manquantes de la colonne VAL_M3: {np.round(nMissingM3/EncodedDataset.shape[0]*100)}% ({nMissingM3})")

In [ ]:
# fill VAL_M3 with 0 ---
EncodedDataset["VAL_M3"].fillna(0, inplace=True)

print("Répartition de la maturité dentaire de la colonne VAL_M3")
EncodedDataset["VAL_M3"].value_counts()

Nous observons qu'après imputation, la maturité dentaire 0 est prédominante. (avant : 1097 zeros, après imputation : 1180 zeros). 

## En utilisant KNN_imputer, remplacer les données manquantes par une prédiction réalisée par l’algorithme des KNN en utilisant les autres features. Vous utiliserez pour cela un voisinage de 5.

### Analyse du renseignement des variables 

D'abord, nous souhaitons connaître le nombre de NA par colonne.

Nous avions déjà remplacer les valeurs manquantes pour la colonne VAL_M3, 1097 valeurs ont été remplacées par 0. VAL_M3 était la variable avec le plus de valeur manquantes.

VAL_PM2 et VAL_I1 sont les colonnes restantes avec le plus de valeurs nulles, 12% pour VAL_PM2 et 11% pour VAL_I1. Toutes les autres colonnes ont moins de 10% de valeurs manquantes. 

Toutes les occurences sont renseignées pour la variable du sexe. 

In [ ]:
# number of NAs ---
naOccurencies=pd.isna(EncodedDataset).sum().reset_index()
naOccurencies.columns= ["Dents","Nombre_NA"]
naOccurencies.sort_values(by="Nombre_NA", inplace=True) # sorting
naOccurencies["PctNA"]=naOccurencies["Nombre_NA"].divide(len(EncodedDataset))

# PLOTTING ------
fig =px.bar(naOccurencies,x= "Dents",y="PctNA" )
fig.update_layout(title= "Nombre de valeurs nulles par colonne")
pio.show(fig, renderer="notebook")

### Imputation des valeurs nulles par *KNN*

Nous proposons une stratégie d'imputation par la méthode des plus proches voisins *KNN*. 

Le *KNNImputer* nous permet de remplacer les valeurs nulles par des valeurs cohérentes en fonction du comportement/score des voisins proches. 

Nous avons choisit, en hyperparamètre, 5 voisins. Cette valeur est souvent choisit pour l'imputation, nous considérons que ce choix est robuste (un bon *trade off*) entre généralisation (réduire la variance) et voisinage (nous voulons que les plus proches, donc, réduire le bias).

Nous avons donc préféré cette méthode plutôt qu'un SimpleImputer (avec la moyenne ou la médianne). 

In [ ]:
# IMPUTER KNN WITH 3 N ------
knnImputer = KNNImputer(n_neighbors=KNNImputerNeighbors)
imputed=knnImputer.fit_transform(EncodedDataset)
# CAST TO DF ------
imputedDF=pd.DataFrame(imputed, columns=EncodedDataset.columns) 

## Séparer le dataset en un dataset d’entrainement [X_train, y_train] et un dataset de test [X_test, y_test]. Le split doit être de 80 / 20.

In [ ]:
X_train, X_test, y_train, y_test= train_test_split(imputedDF,y, test_size= 0.2, random_state=123)

print(f"Nombre de lignes pour entrainement: {len(X_train)}, correspond à {round(len(X_train)/len(imputedDF),2)*100}% du dataset complet")
print(f"Nombre de lignes pour la validation: {len(X_test)}, correspond à {round(len(X_test)/len(imputedDF),2)*100}% du dataset complet")

## Cherchons la meilleure méthode permettant de prédire l’âge en fonction du sexe et de la maturité dentaire.

Pour cela, nous allons utiliser GridSearchCV et Pipeline de la bibliothèque SKLearn pour explorer largement les possibilités de préprocessing (réduction de dimensionnalité) couplées avec quelques algorithmes d’apprentissage supervisés. Tous les entrainements doivent être réalisés sur le dataset d’entrainement et le score doit être comparé au score du pipeline sur le dataset de test.

### a) Créer un pipeline intégrant une PCA et une régression linéaire. Explorer les valeurs du paramètre n_components de la PCA qui donne le meilleur de régression linéaire.

Nous allons utiliser une PCA comme méthode de *preprocessing* avant d'utiliser une regression lineaire pour les prédictions. 

La métrique que nous souhaitons optimiser est la moyenne des erreurs au carré. Nous choisissons les paramètres pour lesquels la moyenne des pertes du *cross validation test set* est la plus basse.

Enfin, avec les meilleurs paramètres, nous évaluons la performance avec le vrai *test set*.

In [ ]:
# PIPELINE ----- 
p1= Pipeline([('PCA', PCA(random_state=123)),('Regression', LinearRegression())])
    #* Params --- 
params= {
    'PCA__n_components': np.arange(1,10).tolist()
    }

# FIT PIPELINE with Gridsearch ---
clf = GridSearchCV(p1, params, # pipeline + params
                   n_jobs=-1, # cpus
                   scoring="neg_root_mean_squared_error")# Scoring 
clf.fit(X_train, y_train);


In [ ]:
    #* Extract results --- 
results= pd.DataFrame(clf.cv_results_)
results.sort_values("mean_test_score",ascending=False, inplace=True)
results["mean_test_score"]=results["mean_test_score"]*-1 # non negative SS

# PLOT RESULTS ------
fig=px.line(results,x='param_PCA__n_components', y="mean_test_score", error_y="std_test_score") 
fig.update_layout(title="Nombre de composants principaux vs CV test score (negative root mean squared error)")
fig.show(renderer="notebook")

Il semblerait que garder 6, 7 ou 8,9 composants principaux aboutit à des résultats très similaires avec la regression linéaire comme prédicteur. 

Nous choissons le meilleur paramètre, qui dans ce cas est 9 composants principaux. Toutefois, ceci présente peu d'avantages parce que nous n'avons pas réussi à réduire la dimensionalité de nos données, donc, nous n'avons pas capturer suffisament de variance à l'aide de moins de variables : 

In [ ]:
print(f"Meilleurs paramètres : {clf.best_params_}")
print(f"Meilleur CV test score : {clf.best_score_}")
print(f"Test Score  (negative Sum of Squares) avec 20% des données qui n'ont pas servi à l'entraînement: {clf.score(X_test, y_test)}") 
print(f"Train Score (negative Sum of Squares) : {clf.score(X_train, y_train)}") 

### b)Créer un pipeline intégrant une PCA et un KNN. En utilisant GridSearch, explorer l’ensemble des couples de paramètres n_components (pour la PCA), n_neighbors (pour KNN) et weights (pour KNN) donnant les meilleurs résultats.

Nous changeons notre algorithme d'apprentissage supervisé, cette fois-ci, nous utilisons un prédicteur par plus proches voisins: 

In [ ]:
# PIPELINE ----- 
p2= Pipeline([('PCA', PCA(random_state=123)),('KNN', KNeighborsRegressor())])
    #* Params --- 
params= {
        'PCA__n_components': np.arange(1,10).tolist(), # PCA 
        'KNN__weights': ["uniform", "distance"], # KNN
        'KNN__n_neighbors': np.arange(1,100) # KNN
        }

# FIT PIPELINE with Gridsearch ---
clf = GridSearchCV(p2, params, n_jobs=-1,scoring="neg_root_mean_squared_error")
clf.fit(X_train, y_train);


Cette fois-ci nous avons plus de paramètres à prendre en compte pour nos choix. 

In [ ]:
    #* Extract results --- 
results= pd.DataFrame(clf.cv_results_)
results.sort_values("mean_test_score",ascending=False, inplace=True)
results["mean_test_score"]=results["mean_test_score"]*-1 # non negative

# PLOT RESULTS ------
fig=px.scatter(results,x= "param_KNN__n_neighbors", y="mean_test_score", symbol="param_KNN__weights",
               color="param_PCA__n_components", 
              size=np.repeat(1,results.shape[0]),
              size_max=12,
              symbol_sequence=["circle-open","arrow-bar-right"])
fig.update_layout(height=500, title ="CV Test Scores from GridSearch. <br>     Form = weight (arrow is 'distance', triangle is 'uniform'). <br>     Color= N_components", margin=dict(t=110))
fig.show(renderer="notebook")

Nous allons examiner les résultats: 

**Nombre de voisins: (axe abscisses)**

Avant 6 ou 7 *nearest neighbors*, l'erreur est élevée quelles que soient les valeurs des autres paramètres (poids des voisins et nombre de composants principaux).

Après 7 voisins, et en fonction des choix des autres paramètres, des bons scores sont possibles. 

**Type de poids des voisins: (triangle ou cercle)**

Nous observons un phénomène qui est assez logique: plus on augmente le nombre de voisins, plus le poids par distance (au lieu d'un poid unique) devient important. 

Visuellement, à partir de 35 plus proches voisins, pondérer les voisins par sa distance euclidienne permet de garder des scores bas, au contraire, continuer à pondérer tous les voisins de la même manière augmente notre score de perte. 

Cependant, aux alentours de 20 ($\pm$ 8) *nearest neighbors*, il paraît qu'un poid uniforme réduit marginalement la perte. 

**Nombre de composants principaux: (couleur)**

Le choix du bon nombre de composants semble difficile. En garder 1 seul composant aboutit a des erreurs élevés quelles que soient les autres paramètres, en effet, ce choix est trop restrictif. 

Toutefois, pour les choix restant (2-9 composants), plusieurs combinaisons de paramètres aboutissent à des bon résultats. Plusieurs fois, 2 composants sont suffisants si la pondération est uniforme et le nombre de voisins n'est pas très élevé (ce choix serait intéressant pour visualiser les données ainsi que pour des questions de rapidité).


Nous choissons les paramètres qui ont permis d'obtenier l'erreur le plus bas, nous considérons si ce choix est en accord à nos observations: 

In [ ]:
print(f"Meilleurs paramètres : {clf.best_params_}")
print(f"Meilleur CV test score : {clf.best_score_}")
print(f"Test Score  (negative Sum of Squares) avec 20% des données qui n'ont pas servi à l'entraînement: {clf.score(X_test, y_test)}") 
print(f"Train Score (negative Sum of Squares) : {clf.score(X_train, y_train)}") 

### c) Ajouter un scaler en début pipeline et tester, en plus des paramètres précédents les 4 fonctions de normalisation StandardScaler, MinMaxScaler, Normalizer et MaxAbsScaler.

Nous évaluons maintenant l'impact de standardiser nos données par plusieurs moyens: 

In [ ]:
# Scalers to try --- 
scalers= ["StandardScaler","MinMaxScaler","Normalizer","MaxAbsScaler"]
resultsList= []
for scaleProcedure in scalers: 
    print(f"------ Fitting Pipeline with {scaleProcedure} ------")
    p2= Pipeline([(scaleProcedure, eval(scaleProcedure)()), # evaluate the scaler string
                  ('PCA',PCA()),
                  ('KNN', KNeighborsRegressor())])

    params= {
            'PCA__n_components': np.arange(1,10).tolist(),
            'KNN__weights': ["uniform", "distance"],
            'KNN__n_neighbors': np.arange(1,100)
            }

    clf = GridSearchCV(p2, params, n_jobs=-1,scoring="neg_root_mean_squared_error")
    clf.fit(X_train, y_train)
    
    # Results ---
    results= pd.DataFrame(clf.cv_results_)
    results.sort_values("mean_test_score",ascending=False, inplace=True)
    results["mean_test_score"]=results["mean_test_score"]*-1 
    results["ScalerUsed"]= scaleProcedure
    
    # Save them --- 
    resultsList.append(results)

Nous avons de nombreuses combinaisons (1782 (9\*2\*99) paramètres pour chaque fonction de standardisation, donc 1782\*4). Nous choisissons de visualiser uniquement les 100 combinaisons qui ont aboutit aux meilleurs résultats du *cross validation test set*.

In [ ]:
# CONCAT AND FILTER BEST 100 ------ 
top100params=pd.concat(resultsList).query("rank_test_score<100")
top100params.reset_index(drop=True, inplace=True)

Cette fois-ci, nous ne visualisons pas les paramètres du nombre de composants ou du type de pondération des voisins. 

Par contre, nous affichons chaque méthode de standardisation à l'aide de couleurs. 

In [ ]:
# PLOT RESULTS -----
fig=px.scatter(top100params,x="param_KNN__n_neighbors",  y ="mean_test_score" ,color="ScalerUsed") #, error_y="std_test_score" ) 
fig.show(renderer="notebook")

Nous pouvons observer directement que pour toutes les combinaisons de paramètres, la méthode *Normalizer* a produit des erreurs plus élevées que les autres algorithmes (la norme l2 est utilisée par defaut).

Ensuite, le *MaxAbsScaler*, même si compétitif, produit des erreurs moins avantageux que les deux autres méthodes. 

Enfin, le StandardScaler et le MinMaxScaler aboutissent aux meilleurs scores, notamment, avec un nombre de voisins compris entre 8 et 17. 

Voyons les meilleurs paramètres obtenus: 

In [ ]:
# Extract best params ---
best =top100params.iloc[np.argmin(top100params["mean_test_score"]),:]
bestKnnNeighbors= best["param_KNN__n_neighbors"]
bestKnnWeights= best["param_KNN__weights"]
bestPcaNComponents= best["param_PCA__n_components"]
bestScaler= best["ScalerUsed"]
bestScore= best["mean_test_score"]

# Refit --- 
BestPipe= Pipeline([(bestScaler, eval(bestScaler)()), # evaluate the scaler string
                  ('PCA',PCA()),
                  ('KNN', KNeighborsRegressor())])
bestParams= {
            'PCA__n_components': bestPcaNComponents,
            'KNN__weights': bestKnnWeights,
            'KNN__n_neighbors': bestKnnNeighbors
            }
BestPipe.set_params(**bestParams) # Use best params founded by GridSearch 
BestPipe.fit(X_train, y_train)
predTrain=BestPipe.predict(X_train) # predictions for training
predTest=BestPipe.predict(X_test) # predictions for testing 

# SCORE ---
scoreTrain=np.sqrt(mean_squared_error(predTrain, y_train))
scoreTest=np.sqrt(mean_squared_error(predTest, y_test))

# Print best results ---
print(f"Meilleurs paramètres : N_Neighbors : {bestKnnNeighbors}, Weight: {bestKnnWeights}, N_components: {bestPcaNComponents}, Scaler: {bestScaler}")
print(f"Meilleur CV test score : {bestScore}")
print(f"Test Score  (negative Sum of Squares) avec 20% des données qui n'ont pas servi à l'entraînement: {scoreTest}") 
print(f"Train Score (negative Sum of Squares) : {scoreTrain}") 

Utiliser une méthode de standardisation a amélioré nos scores.

D'une part, nous avons réduit la dimensionalité de nos données mais pas drastiquement (le pipeline précédent en avait gardé que 2 composants, le premier pipeline en avait garder 9!). D'autre part, le test score par *cross validation* et très proche du train score. 

**Comparaison de entre Pipelines**

Nous pouvons donc comparer nos scores pour le train set (20% n'ayant pas été utilisé pour l'entraînement) entre pipelines: 
- PCA + regressions linéaire: Meilleur Test Score 2.6592
- PCA + Knnregressor : Meilleur Test Score : 1.9001
- MinMaxScaler + PCA + Knnregressor : Meilleur Test Score : 1.8577

Donc, nous avons améliore notre pouvoir prédictif à chaque étape, notamment, grâce à la recherche de paramètres.

### Analyse des erreurs

Notre modèle, même s'il est simple (15 *neighbors*), obtient un bon score (root mean squared error) de 1.85, il fait donc des erreurs peu considérables, en moyenne. 

Toutefois, l'erreur de prédiction n'est pas le même pour toutes les âges, en effet, nos prédictions ont plus de variabilité lorsqu'il s'agit d'individu âgés. A l'inverse, les erreurs sont faibles pour les individus jeunes. Ceci est peut-être du à la distribution de l'âge dans notre dataset, en effet, des âges élevés sont sous-représentés par rapport aux jeunes patients. 

In [ ]:
# TEST ERRORS vs GROUND TRUTH ------
preds= pd.DataFrame({"Truth":y_test, "Pred": predTest}).reset_index() # PREDICTIONS TEST 
fig= px.scatter(preds, x="Truth", y= "Pred")
fig.update_layout(title="Pred vs Ground truth, ideal would be an straight line")
fig.show(renderer="notebook")

A nouveau nous voyons que le résidu augmente avec l'âge des patients (résidus en forme de cone). 

De plus, nous observons que les erreurs sont faites à la hausse comme à la baisse, cependant, pour les individus les plus âgés de notre dataset, nous observons que notre modèle prédit constamment un âge trop faible. 

En effet, on peut aussi visualiser cet effet grâce à un sort d'effet de plateau horizontal dans le graphique précedent (les valeurs prédictions n'augementent pas alors que l'âge augmente) ou par l'absence de résidu positif pour les âges les plus élevés dans le graphique suivant.

In [ ]:
# POSITIVE VS NEGATIVE ERRORS VS AGE ------
preds["res"]=preds["Pred"]-preds["Truth"] # residuals
preds["abs_res"]= np.abs(preds["res"]) 
preds["sign_res"]= np.where(preds["res"]<0,"negative","positive") # sign
fig= px.scatter(preds, x="Truth", y= "res",color="sign_res")
fig.update_layout(title="Residuals vs Ground Truth, A larger distance to 0 means a larger error prediction")
fig.show(renderer="notebook")